In [9]:
# imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import pandas as pd 
import numpy as np
from torch.utils.data import Dataset,DataLoader 
import torchvision.models as models

import os
from sklearn.model_selection import train_test_split
from PIL import Image

In [10]:
# setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

df = pd.read_csv('data/images.csv')
# remove the unnecessary columns
df = df.drop(columns=['kids','sender_id'])

# ensure the 'label' column is of type int
classes = sorted(df['label'].unique())
df['label'] = df['label'].map(lambda x: classes.index(x))

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])


In [11]:
class ClothingDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx]['image'] 
        label = self.dataframe.iloc[idx]['label']

        img_path = os.path.join(self.img_dir, f'{img_name}.jpg')  
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

In [12]:
class ResNetClothingClassifier(nn.Module):
    def __init__(self,num_classes):
        super(ResNetClothingClassifier,self).__init__()

        self.resnet = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features,num_classes)
    
    def forward(self,x):
        return self.resnet(x)

In [13]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),  
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet normalization
                         std=[0.229, 0.224, 0.225])
])

train_dataset = ClothingDataset(train_df, 'data/images_compressed', transform=transform)
test_dataset = ClothingDataset(test_df, 'data/images_compressed', transform=transform)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [14]:
def train_model(model, train_loader, test_loader, num_epochs=10, learning_rate=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.1)
    
    best_accuracy = 0.0
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        print(f'Starting epoch {epoch+1}/{num_epochs}')
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
                
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * images.size(0)
        
        # Calculate epoch statistics
        epoch_loss = running_loss / len(train_loader.dataset)
        
        # Validation phase
        model.eval()
        correct = 0
        total = 0
        val_loss = 0.0
        
        with torch.no_grad():
            for images, labels in test_loader:
                images = images.to(device)
                labels = labels.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_loss = val_loss / len(test_loader.dataset)
        accuracy = 100 * correct / total
        scheduler.step(val_loss)
        
        # Print statistics
        print(f'Epoch {epoch+1}/{num_epochs} | '
              f'Train Loss: {epoch_loss:.4f} | '
              f'Val Loss: {val_loss:.4f} | '
              f'Accuracy: {accuracy:.2f}%')
        
        # Save best model
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), 'best_model.pth')
    
    print(f'Training complete. Best accuracy: {best_accuracy:.2f}%')
    return model

In [15]:
num_classes = len(classes)  # Make sure your ClothingDataset has this attribute
model = ResNetClothingClassifier(num_classes=num_classes)

trained_model = train_model(model, train_loader, test_loader, num_epochs=15)


Starting epoch 1/15
Epoch 1/15 | Train Loss: 1.2187 | Val Loss: 1.1556 | Accuracy: 66.85%
Starting epoch 2/15
Epoch 2/15 | Train Loss: 0.7551 | Val Loss: 0.8518 | Accuracy: 70.93%
Starting epoch 3/15
Epoch 3/15 | Train Loss: 0.6337 | Val Loss: 0.7994 | Accuracy: 76.20%
Starting epoch 4/15
Epoch 4/15 | Train Loss: 0.5541 | Val Loss: 0.7805 | Accuracy: 75.65%
Starting epoch 5/15
Epoch 5/15 | Train Loss: 0.4715 | Val Loss: 0.7321 | Accuracy: 77.87%
Starting epoch 6/15
Epoch 6/15 | Train Loss: 0.3823 | Val Loss: 0.7802 | Accuracy: 77.13%
Starting epoch 7/15
Epoch 7/15 | Train Loss: 0.3497 | Val Loss: 0.8799 | Accuracy: 76.57%
Starting epoch 8/15
Epoch 8/15 | Train Loss: 0.3199 | Val Loss: 0.8471 | Accuracy: 76.85%
Starting epoch 9/15
Epoch 9/15 | Train Loss: 0.1676 | Val Loss: 0.6322 | Accuracy: 81.76%
Starting epoch 10/15
Epoch 10/15 | Train Loss: 0.1077 | Val Loss: 0.5840 | Accuracy: 82.78%
Starting epoch 11/15
Epoch 11/15 | Train Loss: 0.0802 | Val Loss: 0.5942 | Accuracy: 82.13%
Starti